In [1]:
import pandas as pd
import json
from collections import defaultdict
import re
import numpy as np
from utilities import Tiempo
import dataExtraction
import statistics

In [2]:
def extraerTiemposEntreInteracciones(rawData):
    ultimaInteraccionJugador = defaultdict()
    
    tiempoInicio_Interaccion_Jugador = defaultdict(defaultdict) #Tiempo entre inicio de nivel y la siguiente interaccion (reinicio, completado o interaccion con tarjeta)
    tiempoInteraccion_Completado_Jugador = defaultdict(defaultdict) #Tiempo entre una interaccion (inicio, reinicio o interaccion con tarjeta) y terminar el nivel
    tiempoInteraccion_Interaccion_Jugador = defaultdict(defaultdict) #Tiempo entre una interaccion con tarjeta y otra interaccion con tarjeta
    
    anomalias = []
    
    erInteracted = re.compile(r'\binteracted$\b')
    erGameObject = re.compile(r'\bgame-object$\b')
    erLevelExitButton = re.compile(r'\blevel_exit_button$\b') 
    erInitialized = re.compile(r'\binitialized$\b')
    erSeriousGame = re.compile(r'\bserious-game$\b')    
    erCompleted = re.compile(r'\bcompleted$\b')
    erLevel = re.compile(r'\blevel$\b')
    erIdLevel = re.compile(r'/')
        
    for e in rawData:
        verb = e["verb"]["id"]
        obj = e["object"]["definition"]["type"]
        objectId = e["object"]["id"]
        name = e["actor"]["name"]
        
        levelCode = erIdLevel.split(objectId)[-1]
        if(levelCode != "editor_level"):
            
            if erSeriousGame.search(obj) and erInitialized.search(verb): #Si inicia el juego se reinicia su ultima interaccion
                if name in ultimaInteraccionJugador:
                    del ultimaInteraccionJugador[name]

            elif erLevel.search(obj):
                if erInitialized.search(verb): #Significa que ha iniciado o reiniciado el nivel
                    timestamp = e["timestamp"]

                    if "result" in e: #Inicia el nivel desde el menu
                        ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "inicio"}
                    else: #Reinicia el nivel
                        try:
                            t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                            
                            if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                #Añadimos la dif de tiempo al diccionario del jugador
                                if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode].append({"timestamp" : t, "accion" : "reinicio"})
                                else:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"timestamp" : t, "accion" : "reinicio"}]
                            #Actualizamos ultima interaccion
                            ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "reinicio"}
                        except:
                            #Si entra aqui es porque ha reiniciado un nivel que no habia iniciado
                            #Trazas estan mal
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha reiniciado un nivel que no estaba iniciado"})

                elif erCompleted.search(verb) and e["result"]["score"]["raw"] != 0:
                    #Si completa un nivel o sale al menu se reinicia su ultima interaccion, si falla el intento pero vuelve a intentarlo no se reinicia
                    #Si raw == 0 significa que se reinicia el nivel
                    timestamp = e["timestamp"]
                    try:
                        t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)                        
                        #Añadimos la dif de tiempo al diccionario del jugador
                        if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                #Añadimos la dif de tiempo al diccionario del jugador
                            if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                tiempoInicio_Interaccion_Jugador[name][levelCode].append({"timestamp" : t, "accion" : "terminado", "exito" : e["result"]["success"]})
                            else:
                                tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"timestamp" : t, "accion" : "terminado", "exito" : e["result"]["success"]}]
                                
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"timestamp" : t, "accion_anterior" : "inicio"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"timestamp" : t, "accion_anterior" : "inicio"}]
                                    
                        elif ultimaInteraccionJugador[name]["accion"] == "reinicio":
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"timestamp" : t, "accion_anterior" : "reinicio"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"timestamp" : t, "accion_anterior" : "reinicio"}]

                        elif ultimaInteraccionJugador[name]["accion"] == "interaccion":
                            if levelCode in tiempoInteraccion_Completado_Jugador[name]:
                                tiempoInteraccion_Completado_Jugador[name][levelCode].append({"timestamp" : t, "accion_anterior" : "interaccionTarjeta"})
                            else:
                                tiempoInteraccion_Completado_Jugador[name][levelCode] = [{"timestamp" : t, "accion_anterior" : "interaccionTarjeta"}]
                        #Actualizamos ultima interaccion, la borramos
                        del ultimaInteraccionJugador[name]
                    except:
                        #Si entra aqui es porque se ha completado un nivel que no se habia iniciado
                        #Probablemente por anomalias en las trazas
                        anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo": "Se ha completado un nivel que no estaba iniciado"})

            elif erGameObject.search(obj) and erInteracted.search(verb):
                if "result" in e and "extensions" in e["result"] and "level" in e["result"]["extensions"]:
                    levelCode = e["result"]["extensions"]["level"]
                    if not erLevelExitButton.search(objectId) and levelCode != "editor_level":
                        timestamp = e["timestamp"]
                        if name in ultimaInteraccionJugador:
                            #e["result"]["extensions"]["action"] <-- Que accion realiza
                            t = Tiempo(ultimaInteraccionJugador[name]["t"], timestamp)
                            #Añadimos la dif de tiempo al diccionario del jugador
                            
                            if ultimaInteraccionJugador[name]["accion"] == "inicio":
                                if levelCode in tiempoInicio_Interaccion_Jugador[name]:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode].append({"timestamp" : t, "accion" : "interaccionTarjeta"})
                                else:
                                    tiempoInicio_Interaccion_Jugador[name][levelCode] = [{"timestamp" : t, "accion" : "interaccionTarjeta"}]
                                    
                            if ultimaInteraccionJugador[name]["accion"] == "interaccionTarjeta":
                                if levelCode in tiempoInteraccion_Interaccion_Jugador[name]:
                                    tiempoInteraccion_Interaccion_Jugador[name][levelCode].append({"timestamp" : t, "accion_anterior" : "interaccionTarjeta"})
                                else:
                                    tiempoInteraccion_Interaccion_Jugador[name][levelCode] = [{"timestamp" : t, "accion_anterior" : "interaccionTarjeta"}]
                                
                            ultimaInteraccionJugador[name] = {"t" : timestamp, "accion" : "interaccion"}
                            
                        else:
                            anomalias.append({"Jugador" : name, "Timestamp" : timestamp, "Nivel" : levelCode, "Motivo" : "Se ha interactuado en un nivel que no estaba iniciado"})
                        
                    
    return {"tiempoInicio_Interaccion_Jugador" : tiempoInicio_Interaccion_Jugador, "tiempoInteraccion_Completado_Jugador" : tiempoInteraccion_Completado_Jugador, "tiempoInteraccion_Interaccion_Jugador" : tiempoInteraccion_Interaccion_Jugador,"anomalias" : anomalias}

In [3]:
def get_Tiempos_Ordenados(tiemposInteracciones):
    tiempos = defaultdict()
    
    for name in tiemposInteracciones:
        for level in tiemposInteracciones[name]:
            for t in tiemposInteracciones[name][level]:
                if level in tiempos:
                    tiempos[level].append(t)
                else:
                    tiempos[level] = [t]
                    
    for t in tiempos:
        tiempos[t].sort(key=lambda traza: traza["timestamp"])
        
    return tiempos

In [11]:
#Devuelve un diccionario con clave nivel y valor el tiempo a partir del cual se considera outlier
def getLimiteOutliers(tiemposNivel):
    limiteOutliers = defaultdict()
    
    for level in tiemposNivel:
        valores = [x["timestamp"] for x in tiemposNivel[level]] #Pasamos los timestamps a un array para calcular los cuartiles
        q1 = np.quantile(valores, 0.25) #Primer cuartil
        q3 = np.quantile(valores, 0.75) #Tercer cuartil
        limiteOutliers[level] = (q3-q1)*1.5 #Tiempo donde empieza a considerarse outlier
    
    return limiteOutliers

In [5]:
pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = None

In [6]:
JSONFile = open('trazasOrdenadas.json')
rawData = json.load(JSONFile)
#verTrazas(rawData)
result = extraerTiemposEntreInteracciones(rawData)
JSONFile.close()

In [ ]:
tiempos_Inicio_Interaccion = get_Tiempos_Ordenados(result["tiempoInicio_Interaccion_Jugador"])
tiempos_Interaccion_Completado = get_Tiempos_Ordenados(result["tiempoInteraccion_Completado_Jugador"])
tiempos_Interaccion_Interaccion = get_Tiempos_Ordenados(result["tiempoInteraccion_Interaccion_Jugador"])

outliers_Inicio_Interaccion = getLimiteOutliers(tiempos_Inicio_Interaccion)
outliers_Interaccion_Completado = getLimiteOutliers(tiempos_Interaccion_Completado)
outliers_Interaccion_Interaccion = getLimiteOutliers(tiempos_Interaccion_Interaccion)

In [ ]:
pd.DataFrame(result["anomalias"])
#Se ha interactuado en un nivel que no estaba iniciado -> Probablemente sea porque completa el nivel, minimiza la ventana de siguiente nivel o menu y se pone a mover las tarjetas

In [ ]:
def verTrazas(rawData):
    for e in rawData:
        name = e["actor"]["name"]
        if(name == "auvqa"):
            print(e)
            print("--------")

In [ ]:
tiempos = [Tiempo("1h/30m/15s"), Tiempo("1h/30m"), Tiempo("1h"), Tiempo("30m/17s"), Tiempo("30m"), Tiempo("28m"), Tiempo("1m/13s"), Tiempo("45s")]
tiempos.sort()
print(tiempos)

In [ ]:
Tiempo(str(50 - 30) + "s")